In [1]:
import os
import re
import time
import numpy as np
from deepface import DeepFace
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import shutil
import random

In [11]:
# Cambia esta ruta a donde tengas las carpetas de LFW
DATASET_PATH = "dataset\\lfw\\lfw-deepfunneled\\lfw-deepfunneled"

def contar_sujetos_con_mas_de_10_fotos_por_carpeta(dataset_path, min_fotos=10):
    sujetos = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
    sujetos_con_mas_de_min_fotos = 0
    for sujeto in sujetos:
        fotos = [f for f in os.listdir(os.path.join(dataset_path, sujeto)) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if len(fotos) >= min_fotos:
            sujetos_con_mas_de_min_fotos += 1
    total_sujetos = len(sujetos)
    return sujetos_con_mas_de_min_fotos, total_sujetos


sujetos_10omas, total = contar_sujetos_con_mas_de_10_fotos_por_carpeta(DATASET_PATH, min_fotos=10)
print(f"Sujetos con 10 o más fotos: {sujetos_10omas} / {total} ({100*sujetos_10omas/total:.2f}%)")

Sujetos con 10 o más fotos: 158 / 5749 (2.75%)


In [10]:
# Cambia esta ruta a donde tengas las imágenes de LFW
DATASET_PATH = "dataset/cplfw/images"

def contar_sujetos_con_mas_de_3_fotos(dataset_path, min_fotos=3):
    regex = re.compile(r"(.+?)_\d+\.jpg$")
    sujetos = {}
    for file in os.listdir(dataset_path):
        match = regex.match(file)
        if match:
            nombre = match.group(1)
            sujetos[nombre] = sujetos.get(nombre, 0) + 1
    total_sujetos = len(sujetos)
    sujetos_con_mas_de_min_fotos = sum(1 for count in sujetos.values() if count >= min_fotos)
    return sujetos_con_mas_de_min_fotos, total_sujetos

sujetos_3omas, total = contar_sujetos_con_mas_de_3_fotos(DATASET_PATH, min_fotos=3)
print(f"Sujetos con 3 o más fotos: {sujetos_3omas} / {total} ({100*sujetos_3omas/total:.2f}%)")

Sujetos con 3 o más fotos: 3811 / 3929 (97.00%)


In [12]:
LFW_PATH = "dataset/lfw/lfw-deepfunneled/lfw-deepfunneled"
CPLFW_PATH = "dataset/cplfw/images"
OUTPUT_PATH = "dataset/merged_lfw_cplfw"

def obtener_imagenes_lfw(lfw_path, min_fotos=10):
    sujetos_validos = {}
    for sujeto in os.listdir(lfw_path):
        path_sujeto = os.path.join(lfw_path, sujeto)
        if os.path.isdir(path_sujeto):
            fotos = sorted([f for f in os.listdir(path_sujeto) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            if len(fotos) >= min_fotos:
                sujetos_validos[sujeto] = fotos[:10]  # exactamente 10, las primeras en orden
    return sujetos_validos

def obtener_imagenes_cplfw(cplfw_path, sujetos_validos):
    regex = re.compile(r"(.+?)_(\d+)\.jpg$")
    sujetos_cplfw = {}
    for file in os.listdir(cplfw_path):
        match = regex.match(file)
        if match:
            nombre = match.group(1)
            idx = int(match.group(2))
            if nombre in sujetos_validos and idx >= 2:
                sujetos_cplfw.setdefault(nombre, []).append(file)
    # Solo nos quedamos con sujetos que tengan al menos una imagen >=_2
    sujetos_cplfw = {k: v for k, v in sujetos_cplfw.items() if len(v) > 0}
    return sujetos_cplfw

def merge_lfw_cplfw(lfw_path, cplfw_path, min_lfw=10):
    sujetos_lfw = obtener_imagenes_lfw(lfw_path, min_lfw)
    sujetos_cplfw = obtener_imagenes_cplfw(cplfw_path, sujetos_lfw)
    sujetos_comunes = set(sujetos_lfw.keys()) & set(sujetos_cplfw.keys())
    sujetos_lfw = {k: sujetos_lfw[k] for k in sujetos_comunes}
    sujetos_cplfw = {k: sujetos_cplfw[k] for k in sujetos_comunes}
    return sujetos_lfw, sujetos_cplfw

def copiar_imagenes_a_nueva_carpeta(lfw, cplfw, lfw_path, cplfw_path, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    for sujeto in lfw:
        out_sujeto_path = os.path.join(output_path, sujeto)
        if not os.path.exists(out_sujeto_path):
            os.makedirs(out_sujeto_path)
        # Copia imágenes base (LFW)
        for img in lfw[sujeto]:
            src = os.path.join(lfw_path, sujeto, img)
            dst = os.path.join(out_sujeto_path, "lfw_" + img)
            shutil.copy2(src, dst)
        # Copia imágenes de test (CPLFW)
        for img in cplfw[sujeto]:
            src = os.path.join(cplfw_path, img)
            dst = os.path.join(out_sujeto_path, "cplfw_" + img)
            shutil.copy2(src, dst)

lfw, cplfw = merge_lfw_cplfw(LFW_PATH, CPLFW_PATH)
print(f"Copiando sujetos en común: {len(lfw)}")
copiar_imagenes_a_nueva_carpeta(lfw, cplfw, LFW_PATH, CPLFW_PATH, OUTPUT_PATH)
print(f"¡Listo! Las imágenes se copiaron en {OUTPUT_PATH}")

Copiando sujetos en común: 135
¡Listo! Las imágenes se copiaron en dataset/merged_lfw_cplfw


In [3]:
LFW_PATH = "dataset/lfw/lfw-deepfunneled/lfw-deepfunneled"
CPLFW_PATH = "dataset/cplfw/images"
OUTPUT_PATH = "dataset/merged_lfw_cplfw_50"
N_SUJETOS = 50
RANDOM_SEED = 42

def obtener_imagenes_lfw(lfw_path, min_fotos=10):
    sujetos_validos = {}
    for sujeto in os.listdir(lfw_path):
        path_sujeto = os.path.join(lfw_path, sujeto)
        if os.path.isdir(path_sujeto):
            fotos = sorted([f for f in os.listdir(path_sujeto) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            if len(fotos) >= min_fotos:
                sujetos_validos[sujeto] = fotos[:10]
    return sujetos_validos

def obtener_imagenes_cplfw(cplfw_path, sujetos_validos):
    regex = re.compile(r"(.+?)_(\d+)\.jpg$")
    sujetos_cplfw = {}
    for file in os.listdir(cplfw_path):
        match = regex.match(file)
        if match:
            nombre = match.group(1)
            idx = int(match.group(2))
            if nombre in sujetos_validos and idx >= 2:
                sujetos_cplfw.setdefault(nombre, []).append(file)
    sujetos_cplfw = {k: v for k, v in sujetos_cplfw.items() if len(v) > 0}
    return sujetos_cplfw

def merge_lfw_cplfw(lfw_path, cplfw_path, min_lfw=10):
    sujetos_lfw = obtener_imagenes_lfw(lfw_path, min_lfw)
    sujetos_cplfw = obtener_imagenes_cplfw(cplfw_path, sujetos_lfw)
    sujetos_comunes = set(sujetos_lfw.keys()) & set(sujetos_cplfw.keys())
    sujetos_lfw = {k: sujetos_lfw[k] for k in sujetos_comunes}
    sujetos_cplfw = {k: sujetos_cplfw[k] for k in sujetos_comunes}
    return sujetos_lfw, sujetos_cplfw

def copiar_imagenes_a_nueva_carpeta(lfw, cplfw, lfw_path, cplfw_path, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    for sujeto in lfw:
        out_sujeto_path = os.path.join(output_path, sujeto)
        if not os.path.exists(out_sujeto_path):
            os.makedirs(out_sujeto_path)
        for img in lfw[sujeto]:
            src = os.path.join(lfw_path, sujeto, img)
            dst = os.path.join(out_sujeto_path, "lfw_" + img)
            shutil.copy2(src, dst)
        for img in cplfw[sujeto]:
            src = os.path.join(cplfw_path, img)
            dst = os.path.join(out_sujeto_path, "cplfw_" + img)
            shutil.copy2(src, dst)

lfw, cplfw = merge_lfw_cplfw(LFW_PATH, CPLFW_PATH)
sujetos_comunes = list(lfw.keys())
print(f"Sujetos en común antes del muestreo: {len(sujetos_comunes)}")
# Selección aleatoria reproducible
random.seed(RANDOM_SEED)
if len(sujetos_comunes) < N_SUJETOS:
    print(f"¡Advertencia! Solo hay {len(sujetos_comunes)} sujetos. Usando todos.")
    sujetos_seleccionados = sujetos_comunes
else:
    sujetos_seleccionados = random.sample(sujetos_comunes, N_SUJETOS)
# Filtra los diccionarios para solo esos sujetos
lfw = {k: lfw[k] for k in sujetos_seleccionados}
cplfw = {k: cplfw[k] for k in sujetos_seleccionados}
print(f"Copiando sujetos seleccionados: {len(lfw)}")
copiar_imagenes_a_nueva_carpeta(lfw, cplfw, LFW_PATH, CPLFW_PATH, OUTPUT_PATH)
print(f"¡Listo! Las imágenes de 50 sujetos seleccionados se copiaron en {OUTPUT_PATH}")

Sujetos en común antes del muestreo: 135
Copiando sujetos seleccionados: 50
¡Listo! Las imágenes de 50 sujetos seleccionados se copiaron en dataset/merged_lfw_cplfw_50
